In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

### Q1,2,3

In [3]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    print('Original shape', df.shape)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    print('std', df.duration.std())
    
    or_len = len(df)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    print("dropped outliers", len(df)/or_len)
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

Original shape (3066766, 19)
std 42.59435124195458
dropped outliers 0.9812202822125979
Original shape (2913955, 19)
std 42.84210176105113
dropped outliers 0.9800944077722545


In [5]:
categorical = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

### Q4

In [6]:
X_train.shape

(3009173, 515)

In [7]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

### Q5

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred)**0.5

TypeError: got an unexpected keyword argument 'squared'

### Q6

In [11]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)**0.5

7.811818743246608